In [1]:
import torch 
import numpy as np
import pandas as pd

In [16]:
inputs = torch.tensor([1,2,3])
weights = torch.arange(9).reshape((3,3))
biases = torch.tensor([1,2,3])

def layer(inputs, weights, biases):
    """Pure python NN Layer

    Args:
        inputs (list): inputs from previous layer
        weights (list): weights of each neuron in layer
        biases (list): biases of each neuron in layer

    Returns:
        list: output of each neuron in layer 
    """
    layer_outputs = []    
    for neuron_weights, neuron_bias in zip(weights, biases):
        neuron_output=0
        for n_input, weight in zip(inputs, neuron_weights):
            neuron_output += n_input*weight
        neuron_output +=neuron_bias
        layer_outputs.append(neuron_output)
    return layer_outputs

layer(inputs, weights, biases)

[tensor(9), tensor(28), tensor(47)]

In [14]:
def layer_vec(inputs, weights, biases):
    """PyTorch vectorized NN Layer

    Args:
        inputs (torch.tensor): inputs from previous layer
        weights (torch.tensor): weights of each neuron in layer
        biases (torch.tensor): biases of each neuron in layer

    Returns:
        torch.tensor: output of each neuron in layer 
    """
    cummulative = torch.matmul(weights, inputs) + biases
    return cummulative

In [17]:
layer_vec(inputs, weights, biases)

tensor([ 9, 28, 47])